### NIM : A11.2022.14632
### Kelas: STKI-A11.4701
### Nama: Ephesians Prismaranatha

# Recomendation E-Book Recommendation System Based on Opening Chess Weakness Analysis

### Ringkasan
Proyek ini bertujuan untuk mengembangkan sistem rekomendasi e-book yang menyarankan buku berdasarkan analisis kelemahan dalam pembukaan catur. Sistem ini menggunakan teknik pengambilan data untuk menemukan e-book yang relevan dan memberikan rekomendasi yang dapat membantu pemain catur meningkatkan pemahaman dan strategi mereka dalam pembukaan permainan.

### Permasalahan
Permasalahan utama yang ingin diselesaikan oleh proyek ini adalah:
- Bagaimana cara menganalisis permainan catur untuk mengidentifikasi kelemahan pada fase pembukaan?
- Bagaimana cara merekomendasikan e-book yang relevan berdasarkan analisis kelemahan pembukaan catur?

### Tujuan yang Akan Dicapai
Tujuan dari proyek ini adalah:
- Mengembangkan sistem yang dapat menganalisis permainan catur dan mengidentifikasi kelemahan pada fase pembukaan.
- Merekomendasikan e-book yang relevan untuk membantu pemain catur meningkatkan pemahaman mereka tentang pembukaan yang efektif.

### Model / Alur Penyelesaian
Berikut adalah bagan alur penyelesaian proyek:

```mermaid
graph TD;
    A[Mulai] --> B[Pengumpulan Data]
    B --> C[Preprocessing Data]
    C --> D[Ekstraksi Fitur]
    D --> E[Pelatihan Model]
    E --> F[Generasi Rekomendasi]
    F --> G[Selesai]

## 3. Penjelasan Dataset, EDA dan Proses Features Dataset

### Penjelasan Dataset
Dataset yang digunakan dalam proyek ini adalah file PGN yang berisi catatan permainan catur. File PGN dapat diunduh dari berbagai sumber seperti lichess.com atau chess.com. Dataset ini berisi informasi tentang gerakan catur dan hasil permainan.

### Exploratory Data Analysis (EDA)
EDA dilakukan untuk memahami struktur dan karakteristik dataset. Langkah-langkah EDA meliputi:
- Memuat file PGN dan mengurai permainan catur.
- Menghitung jumlah permainan untuk setiap pembukaan.
- Menghitung win rate untuk setiap pembukaan berdasarkan warna.

### Proses Features Dataset
Proses features dataset meliputi:
- Mengurai permainan catur menjadi daftar gerakan dan hasil.
- Mengelompokkan permainan berdasarkan pembukaan.
- Menghitung jumlah permainan dan win rate untuk setiap pembukaan.

## 4. Proses Learning / Modeling

Proses learning atau modeling dalam proyek ini meliputi:
- Memuat file PGN dan mengurai permainan catur.
- Menghitung win rate untuk setiap pembukaan berdasarkan warna.
- Menampilkan hasil analisis dalam bentuk tabel.
- Memberikan rekomendasi pembukaan yang perlu dipelajari lebih lanjut berdasarkan win rate terendah.

## 5. Performa Model

Performa model diukur berdasarkan kemampuan aplikasi untuk:
- Memuat dan mengurai file PGN dengan benar.
- Menghitung win rate untuk setiap pembukaan dengan akurat.
- Menampilkan hasil analisis dalam bentuk yang mudah dipahami oleh pengguna.
- Memberikan rekomendasi pembukaan yang relevan berdasarkan win rate.

## 6. Diskusi Hasil dan Kesimpulan

### Diskusi Hasil
Hasil analisis menunjukkan bahwa aplikasi Chess Analyzer dapat membantu pemain catur mengidentifikasi kelemahan pada fase pembukaan. Aplikasi ini berhasil menghitung win rate untuk setiap pembukaan dan memberikan rekomendasi pembukaan yang perlu dipelajari lebih lanjut.

### Kesimpulan
Proyek Chess Analyzer berhasil mencapai tujuan yang ditetapkan. Aplikasi ini dapat memuat file PGN, menganalisis permainan catur, menghitung win rate untuk setiap pembukaan, dan memberikan rekomendasi pembukaan yang perlu dipelajari lebih lanjut. Aplikasi ini dapat digunakan oleh pemain catur untuk meningkatkan permainan mereka dengan fokus pada pembukaan yang memiliki win rate terendah.
```

## Library
Library berikut digunakan dalam aplikasi ini:
- `tkinter`: Untuk membuat antarmuka pengguna grafis (GUI).
- `tkinter.filedialog`: Untuk operasi dialog file.
- `tkinter.messagebox`: Untuk menampilkan kotak pesan.
- `tkinter.ttk`: Untuk widget bertema.
- `chess.pgn`: Untuk memparsing file PGN (Portable Game Notation).
- `pandas`: Untuk manipulasi dan analisis data.
- `os`: Untuk operasi file dan direktori.
- `webbrowser`: Untuk membuka URL di peramban web.

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import chess.pgn
import pandas as pd
import os
import webbrowser

## Pra-pemrosesan Dataset
1. Ambil langkah-langkah permainan (moves), hasil permainan (result), dan warna pemain (hitam/putih).
2. Pastikan jumlah dataset minimal ada 30 permainan di setiap langkahnya untuk analisis yang optimal.
3. Proses langkah-langkah permainan dengan mengambil 2 langkah awal untuk dijadikan variabel.
4. Jadikan hasil permainan sebagai variabel.
5. Buat win rate dengan menghitung semua kemenangan saat memegang putih dan kemenangan saat memegang hitam.
6. Pisahkan warna hitam dan putih karena mereka adalah kategori yang berbeda.

## Evaluasi
Hasil evaluasi adalah win rate terendah dari kategori putih dan hitam. Win rate terendah menunjukkan pembukaan yang perlu dipelajari lebih lanjut agar permainan menjadi semakin baik.

### Langkah-langkah Membuat Evaluasi:
1. **Analisis Permainan**: Aplikasi akan menganalisis permainan catur dari file PGN yang dipilih.
2. **Menghitung Win Rate**: Win rate dihitung berdasarkan jumlah kemenangan dari setiap pembukaan.
3. **Memisahkan Kategori**: Hasil analisis dipisahkan berdasarkan kategori putih dan hitam.
4. **Menentukan Win Rate Terendah**: Dari hasil analisis, win rate terendah dari masing-masing kategori diidentifikasi.
5. **Menampilkan Hasil Evaluasi**: Hasil evaluasi ditampilkan, menunjukkan pembukaan yang perlu dipelajari lebih lanjut.

### Attributes:
- `root (tk.Tk)`: Root window dari aplikasi.
- `dataset_dir (str)`: Direktori tempat dataset disimpan.
- `white_pgn_label (tk.Label)`: Label untuk menampilkan path file PGN putih yang dipilih.
- `black_pgn_label (tk.Label)`: Label untuk menampilkan path file PGN hitam yang dipilih.
- `tree_white (ttk.Treeview)`: Treeview untuk menampilkan hasil analisis permainan putih.
- `tree_black (ttk.Treeview)`: Treeview untuk menampilkan hasil analisis permainan hitam.
- `white_pgn_path (str)`: Path file PGN putih yang dipilih.
- `black_pgn_path (str)`: Path file PGN hitam yang dipilih.
- `white_analysis (pd.DataFrame)`: DataFrame hasil analisis permainan putih.
- `black_analysis (pd.DataFrame)`: DataFrame hasil analisis permainan hitam.

### Methods:
- `__init__(root)`: Inisialisasi ChessAnalyzerApp dengan root window.
- `create_widgets()`: Membuat dan menempatkan semua widget dalam aplikasi.
- `show_datasets()`: Menampilkan daftar dataset yang tersedia untuk diunduh.
- `select_white_pgn()`: Membuka dialog untuk memilih file PGN putih.
- `select_black_pgn()`: Membuka dialog untuk memilih file PGN hitam.
- `select_dataset()`: Membuka dialog untuk memilih file dataset PGN.
- `analyze()`: Menganalisis file PGN putih dan hitam yang dipilih.
- `load_pgn(pgn_path)`: Memuat file PGN dan mengembalikan daftar permainan.
- `compute_win_rates(data, color)`: Menghitung win rate untuk setiap pembukaan berdasarkan data permainan.
- `display_results()`: Menampilkan hasil analisis dalam treeview.
- `show_tutorial()`: Menampilkan tutorial cara menggunakan aplikasi.
- `open_dataset_link()`: Membuka link Google Drive yang berisi dataset PGN.


In [3]:
import streamlit as st
import chess.pgn
import pandas as pd
import os
import io
import nltk
from nltk.tokenize import word_tokenize
from docx import Document
from PyPDF2 import PdfFileReader
import requests
from bs4 import BeautifulSoup

# Function to download NLTK data
def download_nltk_data():
    with st.spinner('Downloading NLTK data...'):
        nltk.download('punkt')
    st.success('NLTK data downloaded!')

# Function to preprocess text
def preprocess_text(file_content, file_type):
    text = ""
    if file_type == "application/pdf":
        reader = PdfFileReader(file_content)
        for page_num in range(reader.numPages):
            text += reader.getPage(page_num).extract_text()
    elif file_type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
        doc = Document(file_content)
        for para in doc.paragraphs:
            text += para.text
    tokens = word_tokenize(text.lower())
    return tokens

# Function to search in e-books
def search_engine(query):
    matched_files = []
    dataset_url = "https://github.com/EPatha/Information_Retrieval_System/tree/main/STKI-A11.2022.14632-UAS/Dataset/"
    response = requests.get(dataset_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        files = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.pdf') or a['href'].endswith('.docx')]
        for file in files:
            file_url = "https://raw.githubusercontent.com" + file.replace('/blob/', '/')
            response = requests.get(file_url)
            if response.status_code == 200:
                file_type = response.headers['Content-Type']
                with io.BytesIO(response.content) as file_content:
                    tokens = preprocess_text(file_content, file_type)
                    if query in " ".join(tokens):
                        matched_files.append((file_url, file))
    return matched_files

# Function to load PGN files
def load_pgn(uploaded_file):
    games = []
    pgn_content = uploaded_file.read().decode("utf-8")
    pgn_file = io.StringIO(pgn_content)
    while True:
        game = chess.pgn.read_game(pgn_file)
        if game is None:
            break
        moves = [move.uci() for move in game.mainline_moves()]
        result = game.headers["Result"]
        games.append({"Moves": moves, "Result": result})
    return games

# Function to compute win rates
def compute_win_rates(data, color):
    openings = {}
    for game in data:
        if len(game["Moves"]) < 2:
            continue
        opening = " ".join([game["Moves"][0][2:], game["Moves"][1][2:]])
        if opening not in openings:
            openings[opening] = {"games": 0, "wins": 0}
        openings[opening]["games"] += 1
        if (color == "white" and game["Result"] == "1-0") or (color == "black" and game["Result"] == "0-1"):
            openings[opening]["wins"] += 1

    analysis = []
    for opening, stats in openings.items():
        if stats["games"] >= 30:  # Filter to include only openings with 30 or more games
            win_rate = (stats["wins"] / stats["games"]) * 100
            analysis.append({"Opening": opening, "Games": stats["games"], "Win Rate (%)": win_rate})

    return pd.DataFrame(analysis)

def main():
    st.title("Recomendation E-Book Recommendation System Based on Opening Chess Weakness Analysis")

    st.markdown(
        """
        Dataset berada di link ini, download dulu di link Google Drive tersebut:
        [Google Drive Dataset](https://drive.google.com/drive/folders/1_FWcpYO7noxe5gpb_DLPTKnc2PVimuzu?usp=sharing)
        """
        """
        Search E-book muncul saat sudah mendapat hasil winrate terendah dari permainan catur yang telah diupload.
        """
    )

    # Download NLTK data
    if 'nltk_data_downloaded' not in st.session_state:
        download_nltk_data()
        st.session_state['nltk_data_downloaded'] = True

    st.sidebar.title("Options")
    tutorial_button = st.sidebar.button("Tutorial")
    if tutorial_button:
        tutorial_text = (
            "Cara Menggunakan Recomendation E-Book Recommendation System Based on Opening Chess Weakness Analysis:\n"
            "1. Dapatkan file PGN dari permainan catur. \n"
            "   (Klik tombol 'Dataset' untuk membuka link Google Drive yang berisi dataset PGN yang tersedia untuk diunduh.)\n"
            "   Atau\n"
            "   (Anda bisa mendapatkan file PGN pribadi atau orang lain dari situs seperti lichess.com atau chess.com.\n"
            "   Pergi ke situs openingtree.com dan masukkan nickname Anda untuk menganalisis permainan Anda.\n"
            "   Setelah analisis selesai, unduh file PGN dari situs tersebut).\n"
            "2. Klik 'Select White PGN' untuk memilih file PGN permainan Anda sebagai putih.\n"
            "3. Klik 'Select Black PGN' untuk memilih file PGN permainan Anda sebagai hitam.\n"
            "4. Setelah kedua file PGN dipilih, klik tombol 'Analyze'.\n"
            "5. Aplikasi akan menganalisis permainan Anda dan menampilkan hasilnya di tabel.\n"
            "6. Hasil evaluasi akan menunjukkan pembukaan yang perlu Anda pelajari lebih lanjut berdasarkan win rate.\n"
            "7. Klik tombol 'Search Moves in E-books' untuk mencari rekomendasi e-book berdasarkan pembukaan yang perlu dipelajari.\n"
            "8. Aplikasi akan menampilkan daftar e-book yang sesuai dengan pembukaan yang perlu dipelajari.\n"
            "9. Klik link e-book untuk membuka e-book tersebut.\n"
            "10. Selamat membaca!"
        )
        st.info(tutorial_text)

    if 'nltk_data_downloaded' in st.session_state and st.session_state['nltk_data_downloaded']:
        white_pgn_file = st.sidebar.file_uploader("Select White PGN file", type=["pgn"])
        black_pgn_file = st.sidebar.file_uploader("Select Black PGN file", type=["pgn"])

        if st.sidebar.button("Analyze"):
            if white_pgn_file is None or black_pgn_file is None:
                st.error("Please select both White and Black PGN files.")
                return

            white_games = load_pgn(white_pgn_file)
            black_games = load_pgn(black_pgn_file)

            white_analysis = compute_win_rates(white_games, "white")
            black_analysis = compute_win_rates(black_games, "black")

            st.subheader("White Analysis")
            st.dataframe(white_analysis)

            st.subheader("Black Analysis")
            st.dataframe(black_analysis)

            white_least_winrate = white_analysis.loc[white_analysis["Win Rate (%)"].idxmin()]
            black_least_winrate = black_analysis.loc[black_analysis["Win Rate (%)"].idxmin()]

            st.subheader("Hasil Evaluasi")
            st.write(f"Pembukaan Putih yang perlu dipelajari = {white_least_winrate['Opening']} (Win Rate: {white_least_winrate['Win Rate (%)']:.2f}%)")
            st.write(f"Pembukaan Hitam yang perlu dipelajari = {black_least_winrate['Opening']} (Win Rate: {black_least_winrate['Win Rate (%)']:.2f}%)")

            search_query = white_least_winrate['Opening'] if white_least_winrate['Win Rate (%)'] < black_least_winrate['Win Rate (%)'] else black_least_winrate['Opening']
            if st.button("Search Moves in E-books"):
                matched_files = search_engine(search_query)
                if matched_files:
                    st.write("Search Results for Query: '{}':".format(search_query))
                    for file_url, file_name in matched_files:
                        st.write(f"- [{file_name}]({file_url})")
                else:
                    st.write("No matching files found.")
    else:
        st.warning("NLTK data is being downloaded. Please wait...")

if __name__ == "__main__":
    main()

2025-01-19 07:51:56.469 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-19 07:51:56.814 
  command:

    streamlit run /home/ep/.local/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-01-19 07:51:56.816 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-19 07:51:56.816 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-19 07:51:56.818 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-19 07:51:56.819 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-19 07:51:56.820 Session state does not function when running a script without `streamlit run`
2025-01-19 07:51:56.821 Thread 'MainThread': missing ScriptRunContext



### Penjelasan Kode

1. **Import Libraries**:
   Mengimpor pustaka yang diperlukan seperti `streamlit`, `chess.pgn`, `pandas`, `os`, `io`, `nltk`, `docx`, `PyPDF2`, `requests`, dan `BeautifulSoup`.

2. **Download NLTK Data**:
   Fungsi `download_nltk_data` digunakan untuk mengunduh data NLTK dan menampilkan indikator pemuatan.

3. **Preprocess Text**:
   Fungsi `preprocess_text` digunakan untuk membaca dan memproses teks dari file PDF dan DOCX. Teks yang diekstrak kemudian di-tokenisasi menggunakan NLTK.

4. **Search Engine**:
   Fungsi `search_engine` digunakan untuk mencari e-book dari URL GitHub berdasarkan nama file. Fungsi ini mengunduh file e-book, memproses teksnya, dan mencari query yang diberikan.

5. **Load PGN Files**:
   Fungsi `load_pgn` digunakan untuk memuat file PGN dan mengurai permainan catur. Setiap permainan diurai menjadi daftar gerakan dan hasil.

6. **Compute Win Rates**:
   Fungsi `compute_win_rates` digunakan untuk menghitung win rate untuk setiap pembukaan berdasarkan warna. Hanya pembukaan dengan 30 atau lebih permainan yang disertakan dalam analisis.

7. **Main Function**:
   Fungsi `main` digunakan untuk membuat antarmuka pengguna menggunakan Streamlit. Ini termasuk judul, teks informasi tentang link dataset, tombol, pemuat file PGN, dan fitur pencarian. Tombol lain dinonaktifkan selama proses pemuatan data NLTK.

8. **Run Application**:
   Memanggil fungsi `main` untuk menjalankan aplikasi Streamlit.

### Fitur NLTK dan Pencarian E-Book

#### NLTK (Natural Language Toolkit)
NLTK adalah pustaka Python yang menyediakan alat untuk bekerja dengan teks, termasuk tokenisasi, stemming, dan analisis teks lainnya. Dalam proyek ini, NLTK digunakan untuk:
- **Tokenisasi**: Memecah teks menjadi kata-kata atau token individu untuk analisis lebih lanjut.

#### Pencarian E-Book
Fitur pencarian e-book memungkinkan pengguna untuk mencari e-book yang relevan berdasarkan analisis kelemahan pembukaan catur. Prosesnya meliputi:
- **Mengunduh dan Memproses E-Book**: E-book dalam format PDF dan DOCX diunduh dari URL GitHub dan diproses untuk mengekstrak teks.
- **Mencocokkan Query**: Teks yang diekstrak dibandingkan dengan query yang diberikan untuk menemukan e-book yang relevan.
- **Menampilkan Hasil**: Hasil pencarian ditampilkan dalam bentuk daftar link e-book yang cocok dengan query.

### Menjalankan Aplikasi Streamlit

Untuk menjalankan aplikasi Streamlit, gunakan perintah berikut di terminal:

```sh
streamlit run /home/ep/Documents/Github/Information_Retrieval_System/STKI-A11.2022.14632-UAS/Code/App_STKI.py
```

Ini akan membuka aplikasi Streamlit di peramban web Anda, dan Anda dapat menggunakan antarmuka web untuk memuat file PGN, menganalisis permainan catur, melihat hasil evaluasi, dan mencari gerakan dalam e-book berdasarkan hasil evaluasi win rate terendah.
```